## Import Important LIB

In [ ]:
import os
import json
from langchain.chat_models import AzureChatOpenAI #ต้องเป็นAzureChatOpenAI

In [49]:
from azure.ai.inference.models import (
    ChatCompletionsToolDefinition,
    FunctionDefinition,
    UserMessage,
)

## Define Function call > Tools

In [8]:
from langchain.tools import tool

In [ ]:
import requests
@tool
def get_person_detail(person_id):
    """สำหรับการค้นชื่อ,เบอร์โทรติดต่อของลูกค้าจากฐานข้อมูลที่มีจาก id ของuser"""
    BASE_URL = "http://127.0.0.1:8000" #ใช้จริงอาจจะต้องเปลี่ยน URL
    response = requests.get(f"{BASE_URL}/get_customer/{person_id}")
    if response.status_code == 200:
        that_Cus = response.json()
        if that_Cus:
            Cus_Name=that_Cus['Name']
            Cus_tel=that_Cus['Telephone']
        else:
            print("Not found that person")
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return f'ลูกค้าชื่อ {Cus_Name} เบอร์โทรศัพท์คือ {Cus_tel} '

@tool
def get_promotion():
    """ให้โปรโมชั่นที่เหมาะสมกับลูกค้า"""
    #คิดว่าถ้ามีเป็นตารางเก็บโปรโมชั่นแต่ละเดือนน่าจะ practicalกว่า แถมน่าจะ Adjustอะไรได้มากกว่าด้วย 
    #อาจจะรับจำนวนกระปุกที่เหลือ และใช้ url 2 ตัว ,1 คือurlรับวันเดือน ,อีก 1 คือurlค้นหาโปรโมชั่นประจำเดือน ของกลุ่มคนที่มีความเสี่ยงที่แตกต่าง
    return 'ซื้อ 3 แถม 2! ซื้อ 5 ลดทันที 50%! แคปซูลลดโคเลสเตอรอล ลดไขมันในเลือด ปรับสมดุลสุขภาพ ลดความเสี่ยงโรคหัวใจ รีบสั่งซื้อเลย!'

@tool
def store_user_detail():
    """เก็บข้อมูลของลูกค้าลงdatabase"""
    BASE_URL="http://127.0.0.1:8000" #ใช้จริงอาจจะต้องเปลี่ยน URL
    response = requests.post()
    return ''

In [76]:
import requests
from datetime import datetime
from dateutil.relativedelta import relativedelta

# @tool #มันไม่ให้ใช้@Tool ตอนเทส เลยขอcommentไว้ก่อน
def Aproximate_usage(Latest_buy, Cus_Instock):
    """ช่วยในการคำนวนการบริโภคของลูกค้า"""
    #BASE_URL ตัวนี้จะไปเอาวันเดือนปี ปัจจุบันมาคำนวน
    BASE_URL = "https://tools.aimylogic.com/api/now?tz=Asia/Istanbul&format=dd/MM/yyyy"
    response = requests.get(BASE_URL)
    if response.status_code != 200:
        print("Error fetching current date:", response.text)
        return "ไม่สามารถดึงวันที่ปัจจุบันได้"

    output = response.json()
    # print("API Response:", output)  # Debugging

    date_str = output.get('formatted', None) 

    try:
        date_obj = datetime.strptime(date_str, "%d/%m/%Y").date()
        Latest_buy_obj = datetime.strptime(Latest_buy, "%Y-%m-%d").date()
        diff = relativedelta(date_obj, Latest_buy_obj) 
        months_diff = diff.years * 12 + diff.months

        # print(f"Latest Buy: {Latest_buy_obj}, Current Date: {date_obj}, Months Diff: {months_diff}")  
        if months_diff < 2:
            return 'ลูกค้ามีสินค้าคงคลังอยู่'
        if months_diff >= 2 or Cus_Instock <= 4:
            return 'ลูกค้ามีจำนวนคงคลังน้อยกว่า 1 เดือนแล้ว แนะนำโปรโมชั่น'
        else:
            return 'ลูกค้ามีสินค้าคงคลังอยู่'
    
    except Exception as e:
        print("Error in date processing:", str(e))
        return "เกิดข้อผิดพลาดในการคำนวณ"

In [ ]:
get_person_detail('1')
Aproximate_usage("2024-12-25", 6)

<Response [200]>
Paul 0654321987


'ลูกค้ามีสินค้าคงคลังอยู่'

In [ ]:
import requests
from datetime import datetime
BASE_URL="https://tools.aimylogic.com/api/now?tz=Asia/Istanbul&format=dd/MM/yyyy" #รับวันที่วันนี้
response = requests.get(BASE_URL)
if response.status_code == 200:
    output=response.json()
    date_str=output['formatted']
    print(output['formatted'])
    print(type(output['formatted']))
    date_obj = datetime.strptime(date_str, "%d/%m/%Y").date()
    print(date_obj)

19/02/2025
<class 'str'>
2025-02-19


## Tools

In [ ]:
tools=[get_person_detail,get_promotion,calculate_capsule_day,Aproximate_usage]

## Chain

In [57]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.tools.render import format_tool_to_openai_function
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

Convert tools to openai format

In [58]:
functions = [format_tool_to_openai_function(f) for f in tools]

## Model initial

In [59]:
endpoint = "https://models.inference.ai.azure.com"
model_name = "gpt-4o-mini"
token = os.environ["GITHUB_TOKEN"]

Note : การสร้างModel OpenAI จาก Azure ต้องทำตามแบบนี้ พวกparameterจุกจิกพวกนี้หมดเลย

In [124]:
# Azzure เขียนการinit model แบบนี้ไม่ได้/ ลองเปลี่ยนเป็น AzureChatOpenAI ก็ไม่ได้
# model = ChatOpenAI(temperature=0).bind(functions=functions)
model = AzureChatOpenAI(
    openai_api_key=token,
    openai_api_base=endpoint,
    model=model_name,
    openai_api_version="2023-03-15-preview",
    temperature=0
).bind(functions=functions)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{input}"),
])
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/langchain_community/chat_models/azure_openai.py:174: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://models.inference.ai.azure.com to https://models.inference.ai.azure.com/openai.
  warnings.warn(


In [125]:
result = chain.invoke({"input": "what is the weather is sf?"})

In [126]:
result.tool

'get_overall_temp'

In [127]:
result.tool_input

{'location': 'San Francisco'}

In [128]:
result = chain.invoke({"input": "I want more information of book name Wan thong"})
result

AgentActionMessageLog(tool='get_book_detail', tool_input={'book_name': 'Wan thong'}, log="\nInvoking: `get_book_detail` with `{'book_name': 'Wan thong'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"book_name":"Wan thong"}', 'name': 'get_book_detail'}}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 118, 'total_tokens': 135, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_5154047bf2', 'finish_reason': 'function_call', 'logprobs': None}, id='run-f6a330e0-a8b8-4e8e-a19e-c8a139ac71ba-0')])